# Conditional Volatility

Asset price volatility is central to derivatives pricing. It is defined as measure of price variability over certain period of time. In essence, it describes standard deviation of returns. There are different types of volatility: Historical, Implied, Forward. In most cases, we assume volatility to be constant, which is clearly not true and numerous studies have been dedicated to estimate this variable, both in academia and industry. 

**Volatility**

Volatility estimation by statistical means assume equal weights to all returns measured over the period. We know that over 1-day, the mean return is small as compared to standard deviation. If we consider a simple *m*-period moving average, where $\sigma_n$ is the volatility of return on day n, then with $\overline u$ $\approx$ 0, we have <br><br>

$$ \sigma^2_n = \frac 1 m \sum_{i=1}^m u^2_{n-i} $$

where, $u$ is return and $\sigma^2$ is the variance.

**ARCH**

However, any large return within this *n* period will elevate the volatility until it drops out of the sample. Further, we observe volatility is mean reverting and tends to vary about a long term mean. To address this effect, we adopt to the weighting schemes.<br><br>

$$ \sigma^2_n = \gamma \overline \sigma^2 + \sum_{i=1}^m \alpha_i u^2_{n-i} $$

$$ \sigma^2_n = \omega + \sum_{i=1}^m \alpha_i u^2_{n-i} $$

where, $\omega = \gamma \overline \sigma^2$ and weights must sum to 1.

This is known as Autoregressive Conditional Heteroscedastic model. Autoregressive models are a statistical technique involving a regression of lagged values where the model suggests that past values can help forecast future values of the same variable. Within the model, a time series is the dependent variable and lagged values are the independent variables. 

The ARCH model, was originally developed by Robert Engle in 1982 to measure the dynamics of inflation uncertainty. Conditional heteroskedasticity refers to the notion that the next period’s volatility is conditional on the volatility in the current period as well as to the time varying nature of volatility. However, given the volatility dynamics, this model fail to fully capture the persistence of volatility.

**GARCH**

To address the shortcoming, ARCH has been extended to a generalised framework where we add volatility as a forecasting feature by adding previous variance. This method is popularly known as Generalized ARCH or GARCH model.<br><br> 

$$ \sigma^2_n = \omega + \sum_{i=1}^p \alpha_i u^2_{n-i} + \sum_{i=1}^q \beta_i \sigma^2_{n-i} $$

where, $p$ and $q$ are lag length.

GARCH(1,1) is then represented as,

$$ \sigma^2_n = \omega + \alpha u^2_{n-1} + \beta \sigma^2_{n-1} $$

where, $\alpha + \beta < 1$ and $\gamma + \alpha + \beta = 1$ as weight applied to long term variance cannot be negative and $\frac {\omega} {(1-\alpha-\beta)}$ is the long-run variance. 

The GARCH model is a way of specifying the dependence of the time varying nature of volatility. The model incorporates changes in the fluctuations in volatility and tracks the persistence of volatility as it fluctuates around its long-term average and are exponentially weighted. To model GARCH or the conditional volatility, we need to derive $\omega$, $\alpha$, $\beta$ by maximizing the likelihood function.

## ARCH Toolbox
ARCH is one of the popular tools used for financial econometrics, written in Python - with Cython and/or Numba used to improve performance. We will now use `arch_model` to fit our GARCH model using this package.

In [ ]:
# Import libraries
import sys
import os
sys.path.append(os.path.abspath(".."))

import numpy as np
import quantmod.charts
from arch import arch_model 
from utils import query_all_stocks

In [ ]:
# Query stock data from database
df = query_all_stocks()
assets = sorted(['ICICIBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT'])
df = df[assets]
returns = df.pct_change().dropna()

# Select a single stock
stockreturn = returns['ICICIBANK']

In [ ]:
# Mean zero
g1 = arch_model(stockreturn*100, vol='GARCH', mean='Zero', p=2, q=2, dist='Normal')
model = g1.fit()

In [ ]:
# Model output
print(model)

In [ ]:
# Extract the conditional volatility
conditional_volatility = model.conditional_volatility

# Annualize the volatility (assuming 252 trading days)
annualized_volatility = conditional_volatility * np.sqrt(252)

**Plot Conditional Volatility**

In [ ]:
# Plot annualised volatility
(annualized_volatility.to_frame()).iplot(kind="line", title="Annualised Volatility from GARCH(1,1) Model")

---
[Kannan Singaravelu](https://www.linkedin.com/in/kannansi) | Refer [Quantmod](https://kannansingaravelu.com/quantmod/) and [Arch](https://arch.readthedocs.io/en/latest/index.html) for more information.